In [2]:
import numpy as np
import loguru
import time
import json
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [3]:
with open('../configs/elm_configs.json', 'r') as f:
    configs = json.load(f)
num_classes = configs['num_classes']
num_hidden_layers = configs['num_hidden_layers']
input_length = configs['input_length']

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, input_length).astype(np.float32) / 255.0
x_test = x_test.reshape(-1, input_length).astype(np.float32) / 255.0

y_train = to_categorical(y_train, num_classes).astype(np.float32)
y_test = to_categorical(y_test, num_classes).astype(np.float32)

In [5]:
def _mean_squared_error(y, pred):
    return 0.5 * np.mean((y - pred) ** 2)

def _mean_abs_error(y, pred):
    return np.mean(np.abs(y, pred))

def _sigmoid(x):
    return 1. / (1. + np.exp(-x))


def _fourier(x):
    return np.sin(x)


def _hardlimit(x):
    return (x >= 0).astype(int)


def _identity(x):
    return x


def getActivation(name):
    if name not in ['sigmoid', 'fourier', 'hardlimit', 'identity']:
        raise ValueError('Unknown activation function: %s' % name)
    
    return {
        'sigmoid': _sigmoid,
        'fourier': _fourier,
        'hardlimit': _hardlimit,
        'identity': _identity
    }[name]


def getLoss(name):
    if name not in ['mse', 'mae']:
        raise ValueError('Unknown loss function: %s' % name)
    
    return {
        'mse': _mean_squared_error,
        'mae': _mean_abs_error
    }[name]

In [6]:
class ReELM:
    def __init__(self, 
                 num_input_nodes: int, 
                 num_hidden_units: int, 
                 num_out_units: int,
                 C: int=1,
                 activation: str='sigmoid', 
                 loss: str ='mse', 
                 beta_init: np.ndarray=None, 
                 w_init: np.ndarray=None,
                 use_bias: bool=True, 
                 bias_init: np.ndarray=None):
        
        self._num_input_nodes = num_input_nodes
        self._num_hidden_units = num_hidden_units
        self._num_out_units = num_out_units
        self.C = C
        self.I = np.eye(num_hidden_units, num_hidden_units)

        self._activation = getActivation(activation)
        self._loss = getLoss(loss)

        if isinstance(beta_init, np.ndarray):
            self._beta = beta_init
        else:
            self._beta = np.random.uniform(-1., 1., size=(self._num_hidden_units, self._num_out_units))

        if isinstance(w_init, np.ndarray):
            self._w = w_init
        else:
            self._w = np.random.uniform(-1, 1, size=(self._num_input_nodes, self._num_hidden_units))

        if isinstance(bias_init, np.ndarray):
            self._bias = bias_init
        else:
            if use_bias:
                self._bias = np.random.uniform(-1, 1, size=(self._num_hidden_units,))
            else:
                self._bias = np.zeros(shape=(self._num_hidden_units,))

        loguru.logger.debug(f'Beta: {self._beta.shape}')
        loguru.logger.debug(f'Weights: {self._w.shape}')
        loguru.logger.debug(f'Bias: {self._bias.shape}')
    
    def fit(self, X, y, display_time=False):
        H = self._activation(X.dot(self._w) + self._bias)
        
        if display_time:
            start_time = time.time()
            
        # Moore-Penrose pseudo inverse
        H_pinv = np.linalg.pinv(H.T @ H + self.I / self.C)
        
        if display_time:
            stop_time = time.time()
            loguru.logger.debug(f'Pseudo inverse took {stop_time - start_time:.4f} seconds')
        
        self._beta = H_pinv @ H.T @ y
    
    def __call__(self, X):
        H = self._activation(X.dot(self._w) + self._bias)
        return H.dot(self._beta)

    def evaluate(self, X, y):
        pred = self(X)
        loss = self._loss(y, pred)
        acc = np.sum(np.argmax(pred, axis=1) == np.argmax(y, axis=1)) / len(y)
        return loss, acc

In [9]:
# create instance of our model
model = ReELM(
    input_length,
    num_hidden_layers,
    num_classes,
    use_bias=True,
)

2024-04-05 16:07:18.676 | DEBUG    | __main__:__init__:41 - Beta: (2048, 10)
2024-04-05 16:07:18.677 | DEBUG    | __main__:__init__:42 - Weights: (784, 2048)
2024-04-05 16:07:18.678 | DEBUG    | __main__:__init__:43 - Bias: (2048,)


In [10]:
# Train
model.fit(x_train, y_train, display_time=True)

2024-04-05 16:07:29.271 | DEBUG    | __main__:fit:56 - Pseudo inverse took 4.7705 seconds


In [11]:
train_loss, train_acc = model.evaluate(x_train, y_train)
print('train loss: %f' % train_loss)
print('train acc: %f' % train_acc)

train loss: 0.008866
train acc: 0.960133


In [12]:
# Validation
val_loss, val_acc = model.evaluate(x_test, y_test)
print('val loss: %f' % val_loss)
print('val acc: %f' % val_acc)

val loss: 0.009346
val acc: 0.952700


In [ ]:
class KernelELM:
    def __init__(self, 
                 num_input_nodes: int, 
                 num_hidden_units: int, 
                 num_out_units: int,
                 C: int=1,
                 activation: str='sigmoid', 
                 loss: str ='mse', 
                 beta_init: np.ndarray=None, 
                 w_init: np.ndarray=None,
                 use_bias: bool=True, 
                 bias_init: np.ndarray=None):
        
        self._num_input_nodes = num_input_nodes
        self._num_hidden_units = num_hidden_units
        self._num_out_units = num_out_units
        self.C = C
        self.I = np.eye(num_hidden_units, num_hidden_units)

        self._activation = getActivation(activation)
        self._loss = getLoss(loss)

        if isinstance(beta_init, np.ndarray):
            self._beta = beta_init
        else:
            self._beta = np.random.uniform(-1., 1., size=(self._num_hidden_units, self._num_out_units))

        if isinstance(w_init, np.ndarray):
            self._w = w_init
        else:
            self._w = np.random.uniform(-1, 1, size=(self._num_input_nodes, self._num_hidden_units))

        if isinstance(bias_init, np.ndarray):
            self._bias = bias_init
        else:
            if use_bias:
                self._bias = np.random.uniform(-1, 1, size=(self._num_hidden_units,))
            else:
                self._bias = np.zeros(shape=(self._num_hidden_units,))

        loguru.logger.debug(f'Beta: {self._beta.shape}')
        loguru.logger.debug(f'Weights: {self._w.shape}')
        loguru.logger.debug(f'Bias: {self._bias.shape}')
    
    def fit(self, X, y, display_time=False):
        H = self._activation(X.dot(self._w) + self._bias)
        
        if display_time:
            start_time = time.time()
            
        # Moore-Penrose pseudo inverse
        H_pinv = np.linalg.pinv(H.T @ H + self.I / self.C)
        
        if display_time:
            stop_time = time.time()
            loguru.logger.debug(f'Pseudo inverse took {stop_time - start_time:.4f} seconds')
        
        self._beta = H_pinv @ H.T @ y
    
    def __call__(self, X):
        H = self._activation(X.dot(self._w) + self._bias)
        return H.dot(self._beta)

    def evaluate(self, X, y):
        pred = self(X)
        loss = self._loss(y, pred)
        acc = np.sum(np.argmax(pred, axis=1) == np.argmax(y, axis=1)) / len(y)
        return loss, acc